In [1]:
import os
import sys
import math
import rasterio
from rasterio import features
from rasterio.features import rasterize, geometry_mask
from rasterio.transform import from_origin
import fiona
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
import numpy as np

In [2]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)

In [3]:
from constants import CLEAN, DISSOLVED, STUDY_BOUNDARY_PATH, DATA_PATH, HANSEN_LOSSYEAR_FILEPATH, HANSEN_TEN_MASK, DISSOLVED_CLEAN_YEAR

In [4]:
clean = gpd.read_file(CLEAN) # Import the final clean lup_dataset with empty properites included from qgis
dissolved = gpd.read_file(DISSOLVED) # clean was dissolved by year but also keeping features distinct providing limit boundaries of each property by put_id
study_boundary = gpd.read_file(STUDY_BOUNDARY_PATH)

# Clean up of dissolved dataset
Removing slivers, readding a lost property, 

In [ ]:
dissolved = dissolved.drop(columns='fid')

In [ ]:
output_crs = 'EPSG:4326'
output_extent = (-62.64186038139295, -25.354320073574613, -57.14929123970096, -19.287457970745013)
output_resolution = (0.00026949458523585647, -0.00026949458523585647)
study_boundary = study_boundary.to_crs(output_crs)
study_area_bounds = study_boundary.total_bounds

In [ ]:
# Creates a  Polygons based one for each year of 2000 - 2012, this will be used to select the correspoding 10-year pixels.
clean_dissolve = dissolved #clean.dissolve('anho_capa', as_index=False)
# Clean Slivers
clean_dissolve.geometry = clean_dissolve.buffer(25, join_style= 2)
clean_dissolve.geometry = clean_dissolve.buffer(-25, join_style= 2)
clean_dissolve = clean_dissolve.reindex(columns=dissolved.columns)

In [ ]:
# Create an empty GeoDataFrame to store the final processed properties
clean_years = gpd.GeoDataFrame(columns=dissolved.columns)

# Add polygons from the year 2000 to clean_years as the baseline
clean_years = pd.concat([clean_years, clean_dissolve[clean_dissolve['anho_capa'] == '2000']])
for year in range(2001, 2013):  # Loop from 2001 to 2012
    # Get polygons of the current year
    current_year_properties = clean_dissolve[clean_dissolve['anho_capa'] == str(year)]
    
    # Iterate over each polygons of the current year
    for idx, current_property in current_year_properties.iterrows():
        # Subtract geometries of older polygons from the current polygons
        for _, older_property in clean_years.iterrows():
            current_property['geometry'] = current_property['geometry'].difference(older_property['geometry'])
        
        # Append the "cut" current polygons to the clean_years GeoDataFrame
        clean_years = pd.concat([clean_years, current_property.to_frame().T])
clean_years.crs = dissolved.crs


In [ ]:
# For Visual Check in Qgis


'''output_path = os.path.join(DATA_PATH,'processing')


# Create the directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
    # Save the GeoDataFrame as a GeoPackage
# Define the filename for the GeoPackage

filename = os.path.join(output_path, "clean_years.gpkg")
clean_years.to_file(filename, driver="GPKG")'''

In [ ]:
selected_row = clean.loc[clean['put_id'] == 'PUT2238']

selected_row_dissolved = selected_row.dissolve(by = 'anho_capa', as_index = False)
selected_row_dissolved.plot()

In [ ]:
selected_row_dissolved.geometry = selected_row_dissolved.buffer(25, join_style= 2)
selected_row_dissolved.geometry = selected_row_dissolved.buffer(-25, join_style= 2)
selected_row_dissolved = selected_row_dissolved.reindex(columns=dissolved.columns)
selected_row_dissolved = selected_row_dissolved.drop(columns=['put_id', 'fecha_res', 'grupo'])

In [ ]:
# Step 1: Filter out non-polygon geometries
clean_years_polygons = clean_years[clean_years['geometry'].apply(lambda geom: isinstance(geom, Polygon) or isinstance(geom, MultiPolygon))]

# Step 2: Dissolve the filtered GeoDataFrame
dissolved_clean_years = clean_years_polygons.dissolve(by='anho_capa', as_index=False)

dissolved_clean_years = dissolved_clean_years.reindex(columns=dissolved.columns)
dissolved_clean_years = dissolved_clean_years.drop(columns=['put_id', 'fecha_res', 'grupo'])



In [ ]:
dissolved_clean_years = pd.concat([dissolved_clean_years, selected_row_dissolved], ignore_index=True)
# This is the final property limit dataset from here were create the spatial blocking dataset then to raster

In [ ]:
# For Visual Check in Qgis


'''output_path = os.path.join(DATA_PATH,'processing')


# Create the directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
    # Save the GeoDataFrame as a GeoPackage
# Define the filename for the GeoPackage

filename = os.path.join(output_path, "dissolved_clean_years.gpkg")
dissolved_clean_years.to_file(filename, driver="GPKG")'''

In [ ]:
dissolved_clean_years

# Read in point
The above  will reproduce the dissolved_clean_years but to save time will read in the shapefile.

In [ ]:
dissolved_clean_years =  gpd.read_file(DISSOLVED_CLEAN_YEAR)

In [ ]:
dissolved_clean_years

In [ ]:
dissolved_clean_years = dissolved_clean_years.to_crs('EPSG:4326')

In [15]:
import rasterio.mask
from rasterio.mask import mask
import matplotlib.pyplot as plt

In [14]:
# Where files will save, can add subfolders if desired
output_dir = os.path.join(DATA_PATH, 'processed_rasters')
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Ensure CRS match between shapefile and raster
shapes = dissolved_clean_years.to_crs(crs=rasterio.open(HANSEN_TEN_MASK).crs)

In [ ]:
# Load the raster
with rasterio.open(HANSEN_TEN_MASK) as src:
    # Crop the raster with the shapefile
    out_image, out_transform = mask(src, shapes.geometry, crop=False)
    out_meta = src.meta.copy()

    # Update the metadata to reflect the new shape (height, width), transform, and nodata value
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "nodata": -1})

    # Save the cropped raster
    output_raster_path = os.path.join(output_dir, "cropped_hansen_ten_mask.tif")
    with rasterio.open(output_raster_path, "w", **out_meta) as dest:
        dest.write(out_image)
        



In [ ]:

'''cropped_raster_path = os.path.join(output_dir, "cropped_hansen_ten_mask.tif")
with rasterio.open(cropped_raster_path) as src:
    cropped_raster = src.read(1)
    cropped_meta = src.meta

    # Create an initial mask with all values set to -1
    modified_raster = np.full(cropped_raster.shape, -1, dtype=cropped_raster.dtype)

    # Process each polygon
    for index, row in dissolved_clean_years.iterrows():
        year = int(row['anho_capa'])  # Convert year to integer
        polygon = row['geometry']

        # Create a mask for the polygon
        polygon_mask = geometry_mask([polygon], transform=src.transform, invert=True, out_shape=src.shape)

        # Determine the range of years to keep
        start_year = year - 2000 + 1  # Adjust based on your data encoding
        end_year = start_year + 9

        # Apply the mask and modify values within the polygon
        within_polygon = (polygon_mask & (cropped_raster >= start_year) & (cropped_raster <= end_year))
        modified_raster[within_polygon] = cropped_raster[within_polygon]

    # Update the metadata for output
    cropped_meta.update(dtype='int16', nodata=-1)

    # Save the final modified raster
    with rasterio.open('final_modified_raster.tif', 'w', **cropped_meta) as dst:
        dst.write(modified_raster, 1)
'''

In [ ]:

cropped_raster_path = os.path.join(output_dir, "cropped_hansen_ten_mask.tif")
with rasterio.open(cropped_raster_path) as src:
    cropped_raster = src.read(1)
    cropped_meta = src.meta

    # Create an initial mask with all values set to -1
    modified_raster = np.full(cropped_raster.shape, -1, dtype=cropped_raster.dtype)

    # Process each polygon
    for index, row in dissolved_clean_years.iterrows():
        year = int(row['anho_capa'])  # Convert year to integer
        polygon = row['geometry']

        # Create a mask for the polygon
        polygon_mask = geometry_mask([polygon], transform=src.transform, invert=True, out_shape=src.shape)

        # Determine the range of years to keep
        start_year = year - 2000 + 1  # Adjust based on your data encoding
        end_year = start_year + 9

        # Apply the mask and modify values within the polygon
        within_polygon = polygon_mask
        in_year_range = (cropped_raster >= start_year) & (cropped_raster <= end_year)

        # Set values within the year range to their corresponding values
        modified_raster[within_polygon & in_year_range] = cropped_raster[within_polygon & in_year_range]

        # Set values outside the year range but within the polygon to 0
        modified_raster[within_polygon & ~in_year_range] = 0

    # Update the metadata for output
    cropped_meta.update(dtype='int16', nodata=-1)

    # Save the final modified raster
    with rasterio.open('final_modified_raster0.tif', 'w', **cropped_meta) as dst:
        dst.write(modified_raster, 1)


# Vector to Raster

In [5]:
output_crs = 'EPSG:4326'
output_extent = ( -62.6418603813929522, -25.3543200735746126, -57.1492912397009576, -19.2874579707450131)
output_resolution = (0.000269494585235856472, -0.000269494585235856472)
study_boundary = study_boundary.to_crs(output_crs)
#study_area_bounds = study_boundary.total_bounds
study_area_bounds = output_extent

In [6]:
clean = clean.to_crs(output_crs)

In [7]:
print(output_extent)

(-62.64186038139295, -25.354320073574613, -57.14929123970096, -19.287457970745013)


In [8]:
study_area_bounds 

(-62.64186038139295,
 -25.354320073574613,
 -57.14929123970096,
 -19.287457970745013)

In [9]:
def vector_to_raster(input_vector, output_raster, attribute, study_area_bounds, value_mapping, single_value=None, resolution=(abs(0.000269494585235856472), abs(-0.000269494585235856472)), dtype='int16'):
    # Check if input_vector is a GeoDataFrame or a file path
    if isinstance(input_vector, gpd.GeoDataFrame):
        gdf = input_vector
    else:
        # Read the input vector file (GeoPackage or Shapefile) into a GeoDataFrame
        gdf = gpd.read_file(input_vector)
    # Reproject the GeoDataFrame to the desired CRS (EPSG:4326)
    gdf = gdf.to_crs(epsg=4326)

    # Ensure that categorical column is string
    gdf[attribute] = gdf[attribute].astype(str)
   

   # If single_value is None, convert the attribute column to numerical values
    # If single_value is provided, set the attribute column to the provided single_value
    if single_value is None:
        gdf[attribute] = gdf[attribute].replace(value_mapping).astype(dtype)
    else:
        gdf[attribute] = single_value


    # Use the study area bounds to define the dimensions and transform of the output raster
    minx, miny, maxx, maxy = study_area_bounds
    width = int(np.ceil((maxx - minx) / abs(resolution[0])))
    height = int(np.ceil((maxy - miny) / abs(resolution[1])))
    out_transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, width, height)


    # Define the metadata for the output raster file
    out_meta = {
        'driver': 'GTiff',
        'width': 20381,
        'height': 22512,
        'count': 1,
        'dtype': dtype,
        'crs': 'EPSG:4326',
        'transform': out_transform,
        'nodata': -1  # Set the nodata value
    }
    
    # Open the output raster file for writing with the specified metadata
    with rasterio.open(output_raster, 'w', **out_meta) as dst:
        # Create a generator of tuples containing the geometry and attribute value for each feature in the input vector data
        shapes = ((geom, value) for geom, value in zip(gdf['geometry'], gdf[attribute]))
        
        # Burn the geometries and their corresponding attribute values into a raster array
        burned = features.rasterize(
            shapes=shapes,         # The generator of geometry-attribute tuples
            fill=-1,                # The default value for pixels not covered by any geometry
            out_shape=(22512, 20381), # The shape of the output raster array (number of rows and columns)
            transform=out_transform,   # The affine transformation matrix that maps pixel coordinates to the coordinate reference system
            dtype=dtype,            # The data type of the raster array
            default_value=-1    # Set the nodata value
        )
        
        # Write the burned raster array to the output raster file
        dst.write(burned, 1)

In [10]:
# To write text file of value mapping
def write_value_mapping(value_mapping, output_file):
    with open(output_file, 'w') as f:
        for key, value in value_mapping.items():
            f.write(f'{key}: {value}\n')

In [11]:
def process_columns(input_vector, output_dir, study_area_bounds, resolution, columns, value_mapping, single_value=None, file_name=None):
    # Check if input_vector is a GeoDataFrame or a file path
    if isinstance(input_vector, gpd.GeoDataFrame):
        gdf = input_vector
        if file_name is None:
            file_name = "output"  # Set a default file name if none is provided
    else:
        # Read the input vector file (GeoPackage or Shapefile) into a GeoDataFrame
        gdf = gpd.read_file(input_vector)
        if file_name is None:
            file_name = os.path.splitext(os.path.basename(input_vector))[0]

    for column in columns:
        column_output_dir = os.path.join(output_dir, column)
        os.makedirs(column_output_dir, exist_ok=True)

        output_raster = f"{column_output_dir}/{file_name}_{column}_raster.tif"
        vector_to_raster(gdf, output_raster, column, study_area_bounds, value_mapping[column], single_value=single_value, resolution=resolution)


# Make Land Use Plan into Raster

In [12]:
def process_files(input_data, output_dir, study_area_bounds, resolution, columns, single_value=None):
    # Generate a global value mapping from the list of unique values for each column
    global_value_mapping = {column: set() for column in columns}
    
    if isinstance(input_data, str):
        gdf = gpd.read_file(input_data)
    elif isinstance(input_data, gpd.GeoDataFrame):
        gdf = input_data
    else:
        raise ValueError("Invalid input data type. Expected file path or GeoDataFrame.")
    
    for column in columns:
        global_value_mapping[column].update(gdf[column].astype(str).unique())
    
    for column in columns:
        global_value_mapping[column] = {value: idx for idx, value in enumerate(sorted(list(global_value_mapping[column])), 1)}
        print(f"Global value mapping for {column}:")
        for value in global_value_mapping[column]:
            print(f"{value}: {global_value_mapping[column][value]}")
    
    # Write the global value mapping for each column to separate .txt files in the corresponding column folder
    for column in columns:
        column_output_dir = os.path.join(output_dir, column)
        os.makedirs(column_output_dir, exist_ok=True)

        output_value_mapping_file = f"{column_output_dir}/{column}_global_value_mapping.txt"
        write_value_mapping(global_value_mapping[column], output_value_mapping_file)

    # Process the input data with the global value mapping
    process_columns(gdf, output_dir, study_area_bounds, columns=columns, resolution=resolution, value_mapping=global_value_mapping, single_value=single_value)


In [16]:


# Call the process_files function for the clean GeoDataFrame
process_files(clean, output_dir, study_area_bounds, output_resolution, columns = ['grupo'])


Global value mapping for grupo:
AREA_AUTORIZADA: 1
BOSQUES: 2
OTRAS_COBERTURAS: 3
OTRAS_TIERRAS_FORESTALES: 4
unclassified: 5


C:\Users\bsf31\AppData\Local\Temp\ipykernel_21600\3759962778.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf[attribute] = gdf[attribute].replace(value_mapping).astype(dtype)


In [ ]:
# Call the process_files function for the dissolved_clean_years GeoDataFrame
process_files(dissolved_clean_years, output_dir, study_area_bounds, output_resolution, columns = ['anho_capa'])

In [ ]:
process_files(clean, output_dir, study_area_bounds, output_resolution, columns = ['put_id'])
